# Cox, Ross and Rubinstein Binomial Method for Options Pricing
> Python Dynamic Programming and Numpy implementations

In [8]:
import math
import numpy as np
import yfinance as yf
from time import time
from functools import wraps

In [9]:
def calc_runtime(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time()
        result = func(*args, **kwargs)
        end_time = time()
        print(f"{func.__name__}({args, kwargs}) executed in ", end="")
        print(f"{end_time - start_time:.6f} seconds")
        return result
    return wrapper

# Initialize input data

In [ ]:
S0 = 120
K = 100
T = 1.0
r = 0.05
N = 1000000
O_Type = "Call"

u = 1.7
d = 1 / u


# Algorithm / Python For Loop Implementation

In [ ]:
@calc_runtime
def binomial_tree_loop(S0, K, T, r, vol, N):
    pass

# Algorithm / NumPy Vectorized Implementation

# Algorithm / C++ Implementation (Using Python Bindings)

# **Testing**